In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/Users/yitsung/Desktop/MasterThesis/EDM_Function/BinaryClassfication/')
# sys.path.append('/raid/ytchang/MasterThesis/EDM_Function/BinaryClassfication/')
from EDMfunction_noPS import *

import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

In [2]:
###########################################################################################
TICKER = 2330
TP = 10
TARGET = f'y_{TP}'

data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
# data = pd.read_csv('/raid/ytchang/MasterThesis/data/TaiwanStockData_Top100_EMA')
###########################################################################################

In [3]:
# import data #
ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
ticker_data = ticker_data.drop(columns=['ticker'])

# SMA-P/P, binary classification #
ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
ticker_data = ticker_data.dropna().reindex()
ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

origi_data = ticker_data.copy() # Dataframe for validation

# Restore to a dataframe without disclosing future information. #
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(TP)
ticker_data = ticker_data.dropna().reset_index(drop=True)

In [4]:
# Splite Library and Prediction #
Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # the last prediction from Library is 6/30
# Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-07-20')] # just test
Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-10-31')] 

sanity check

In [5]:
# SC_Prediction = Library # sanity check
# SC_ticker_data = pd.concat([Library, SC_Prediction], axis=0, ignore_index=True) # sanity check
# SC_Prediction = SC_ticker_data[len(Library):] # sanity check
# SC_Prediction # sanity check

# EDM pipeline

In [6]:
##############################
KN = 5
EMAX = 10
MAXLAG = 10
THETA_SEQ = [1,2,4,7,11,16,22]
##############################

In [7]:
### Create a dataframe for prediction results. ###
# Date = origi_data['Date'][origi_data['Date']<='2023-06-30'].reset_index(drop=True) # sanity check
# Observations = origi_data[TARGET][origi_data['Date']<='2023-06-30'].reset_index(drop=True) # sanity check
# Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-07-20')].reset_index(drop=True) # just test
# Observations = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-07-20')].reset_index(drop=True) # just test
Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
Observations = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)

EDM_result = pd.DataFrame(Date)
EDM_result['Observations'] = Observations
EDM_result['Predictions'] = None

### Starting from Pred[0] (include searching views for EDM) ###
th=0
Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th) # SC_Prediction
Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)

formatted_columns, train_feature = EDM_FeatureProcessing(data=Lib_Pred_df, target=TARGET)
TargetOED, TargetOED_rho = EDM_TargetOED(data=Lib_Pred_df, target=TARGET, valid_interval=60)
f_selec = EDM_FeatureSelection(data=Lib_Pred_df, target=TARGET, 
                               train_feature=train_feature, TargetOED=TargetOED, 
                               E_max=EMAX)
Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                  max_lag=MAXLAG, th=th)
rs_score = EDM_RandomSimplex(Embed_df=Embed_df, target=TARGET, targetOED=TargetOED, 
                             valid_interval=60, kmax=10000, kn=KN)
dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
result_ls, theta, param = EDM_ModelSelection(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                                             theta_seq=THETA_SEQ, Tp=TP)
model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                           theta=theta, param=param, Tp=TP)

X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1) # when choose "NOT" to delete target feature in step.5 from EDM_ModelSelection()
# X_pred = np.array(ML_df.iloc[-1, :-1]).reshape(1, -1) # when choose to delete target feature in step.5 from EDM_ModelSelection()
y_pred = model.predict(X_pred)
y_pred = y_pred[0]

EDM_result.loc[th, 'Predictions'] = y_pred
# print(f"{EDM_result['Date'][th]}: finished")

### Predict the remaining ###
for th in range(1, len(Prediction)): # SC_Prediction
    Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th) # SC_Prediction
    Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)
    Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
                                      max_lag=MAXLAG, th=th)
    dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
    model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
                               theta=theta, param=param, Tp=TP)

    X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1) # when choose "NOT" to delete target feature in step.5 from EDM_ModelSelection()
    # X_pred = np.array(ML_df.iloc[-1, :-1]).reshape(1, -1) # when choose to delete target feature in step.5 from EDM_ModelSelection()
    y_pred = model.predict(X_pred)
    y_pred = y_pred[0]

    EDM_result.loc[th, 'Predictions'] = y_pred
    # print(f"{EDM_result['Date'][th]}: finished")

ACC = len(EDM_result[EDM_result['Predictions'] == EDM_result['Observations']]) / len(EDM_result['Observations'])
print(f'\n ACC: {ACC}\n')

  Theta     Score                                              Param
0     1  0.498165  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
1     2       0.5  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
2     4  0.498165  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
3     7  0.501835  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
4    11       0.5  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
5    16       0.5  [{'C': 0.1, 'fit_intercept': True, 'intercept_...
6    22  0.520268  [{'C': 1, 'fit_intercept': True, 'intercept_sc...

 ACC: 0.3855421686746988



In [8]:
EDM_result.to_csv(f'{TICKER}_MDRSmap_Tp={TP}_ResultOnce.csv', index=False)
EDM_result.tail(60)

,Date,Observations,Predictions
23,2023-08-04,0,1.0
24,2023-08-07,0,1.0
25,2023-08-08,0,1.0
26,2023-08-09,0,1.0
27,2023-08-10,0,1.0
28,2023-08-11,0,1.0
29,2023-08-14,1,1.0
30,2023-08-15,1,1.0
31,2023-08-16,1,1.0
32,2023-08-17,1,1.0


# Test 5 times on one stock

In [9]:
# ###########################################################################################
# TICKER = 2330
# TP = 10
# TARGET = f'y_{TP}'

# data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
# # data = pd.read_csv('/raid/ytchang/MasterThesis/data/TaiwanStockData_Top100_EMA')
# ###########################################################################################
# ##############################
# KN = 100
# EMAX = 10
# MAXLAG = 20
# THETA_SEQ = [1,2,4,7,11,16,22]
# ##############################

In [10]:
# for num in range(5):

#     ### Import data ###
#     # import data #
#     ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
#     ticker_data = ticker_data.drop(columns=['ticker'])

#     # SMA-P/P, binary classification #
#     ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
#     ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
#     ticker_data = ticker_data.dropna().reindex()
#     ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

#     origi_data = ticker_data.copy() # Dataframe for validation

#     # Restore to a dataframe without disclosing future information. #
#     ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(TP)
#     ticker_data = ticker_data.dropna().reset_index(drop=True)

#     # Splite Library and Prediction #
#     Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # the last prediction from Library is 6/30
#     Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-10-31')] 

#     ### Create whole test dataframe ###
#     whole_result = pd.DataFrame()

#     ### Create a dataframe for prediction results. ###
#     Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
#     Observations = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)

#     EDM_result = pd.DataFrame(Date)
#     EDM_result['Observations'] = Observations
#     EDM_result['Predictions'] = None

#     ### Starting from Pred[0] (include searching views for EDM). ###
#     th=0
#     Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
#     Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)

#     formatted_columns, train_feature = EDM_FeatureProcessing(data=Lib_Pred_df, target=TARGET)
#     TargetOED, TargetOED_rho = EDM_TargetOED(data=Lib_Pred_df, target=TARGET, valid_interval=60)
#     f_selec = EDM_FeatureSelection(data=Lib_Pred_df, target=TARGET, 
#                                    train_feature=train_feature, TargetOED=TargetOED, 
#                                    E_max=EMAX)
#     Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
#                                       max_lag=MAXLAG, th=th)
#     rs_score = EDM_RandomSimplex(Embed_df=Embed_df, target=TARGET, targetOED=TargetOED, 
#                                  valid_interval=60, kmax=10000, kn=KN)
#     dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
#     result_ls, theta, param = EDM_ModelSelection(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
#                                                  theta_seq=THETA_SEQ, Tp=TP)
#     model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
#                             theta=theta, param=param, Tp=TP)

#     X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
#     y_pred = model.predict(X_pred)
#     y_pred = y_pred[0]

#     EDM_result.loc[th, 'Predictions'] = y_pred
#     # print(f"{EDM_result['Date'][th]}: finished")

#     ### Predict the remaining ###
#     for th in range(1, len(Prediction)):
#         Library = UnderSampling(Library=Library, target=TARGET)
#         Lib_Pred_df = ConcateLibPred(Library=Library, Prediction=Prediction, th=th)
#         Lib_Pred_df = DataNormalize(Lib_Pred_df=Lib_Pred_df)
#         Embed_df, ML_df = EDM_CreateNewDF(data=Lib_Pred_df, target=TARGET, f_selec=f_selec, 
#                                           max_lag=MAXLAG, th=th)
#         dmatrix = EDM_WeightedDistanceMatrix(Embed_df=Embed_df, rs_score=rs_score)
#         model = EDM_TrainningModel(ML_df=ML_df, target=TARGET, dmatrix=dmatrix, 
#                                    theta=theta, param=param, Tp=TP)
        
#         X_pred = np.array(ML_df.iloc[-1]).reshape(1, -1)
#         y_pred = model.predict(X_pred)
#         y_pred = y_pred[0]

#         EDM_result.loc[th, 'Predictions'] = y_pred
#         # print(f"{EDM_result['Date'][th]}: finished")

#     ACC = len(EDM_result[EDM_result['Predictions'] == EDM_result['Observations']]) / len(EDM_result['Observations'])
#     print(f'\n {num+1} of 5 finished, ACC: {ACC} \n')

#     whole_result = pd.concat([whole_result, EDM_result], axis=0, ignore_index=True)

# whole_result.to_csv(f'{TICKER}_MDRSmap_Tp={TP}_ResultWhole.csv', index=False)

# Test 5 times splited